In [ ]:
---Source code for the "Three Questions Project" database--

CREATE TABLE [Human Rights Defenders]
(
	[Country Name] VARCHAR (30) PRIMARY KEY,
	[Convention] VARCHAR (50)
		CHECK ([Convention] IN ('ECHR','IACHR','ACHPR'))
);

GO

CREATE TABLE [Case Statistics]
(
	[Case Name] VARCHAR (30) PRIMARY KEY,
	[Respondent] VARCHAR(30) NOT NULL FOREIGN KEY REFERENCES [Human Rights Defenders] ([Country Name]),
	[Reliance on Government Record] VARCHAR (10) NOT NULL 
		CHECK ([Reliance on Government Record] IN ('None','Partial', 'Extended', 'Full')),
	[Material Scope of Factual Dispute] VARCHAR (MAX),
	[Political Incident] VARCHAR (100),
	[Political Dummy] INT,
	[Preliminary Objections] VARCHAR (300)
		CHECK ([Preliminary Objections] IN ('No', 'Yes')),
	[Preliminary Factual Reasoning] VARCHAR (300)
		CHECK ([Preliminary Factual Reasoning] IN ('No', 'Yes')),
	[Preliminary Factual Reasoning Content] VARCHAR (300),
	[Qualitative Factual Dispute] VARCHAR (10)
		CHECK ([Qualitative Factual Dispute] IN ('No', 'Yes')),
	[Qualitative Factual Reasoning] VARCHAR (300)
		CHECK ([Qualitative Factual Reasoning] IN ('No', 'Yes')),
	[Qualitative Factual Reasoning Content] VARCHAR (300),
	[Quantitative Factual Dispute] VARCHAR (10)
		CHECK ([Quantitative Factual Dispute] IN ('No', 'Yes')),
	[Quantitative Factual Reasoning] VARCHAR (300)
		CHECK ([Quantitative Factual Reasoning] IN ('No', 'Yes')),
	[Quantitative Factual Reasoning Content] VARCHAR (300),
	[C&E Factual Dispute] VARCHAR (10)
		CHECK ([C&E Factual Dispute] IN ('No', 'Yes')),
	[C&E Factual Reasoning] VARCHAR (300)
		CHECK ([C&E Factual Reasoning] IN ('No', 'Yes')),
	[C&E Factual Reasoning Content] VARCHAR (300),
	[Material Damage Claimed] FLOAT,
    [Non- Material Damage Claimed] FLOAT,
    [C&E Claimed] FLOAT,
	[Material Damage Award] FLOAT,
	[Non-Material Damage Award] FLOAT,
	[C&E Award] FLOAT,
	[Interest Rate] FLOAT,
	[Votes] INT,
	-- We need this check to happen: CHECK (CAST(SUBSTRING ([Votes], 1, 1) AS INT) != CAST(SUBSTRING ([Votes], 3, 3) AS INT) AND SUBSTRING ([Votes], 2, 2) = '-'),
	[Minority Judges] INT,
	[Texts Violated] INT,
    [Articles Violated] INT,
    [Texts Alleged] INT
		CONSTRAINT [Allegation of violation 2] CHECK ([Texts Alleged] IS NOT NULL),
    [Articles Alleged] INT
		CONSTRAINT [Allegation of violation 1] CHECK ([Articles Alleged] IS NOT NULL)

);

GO

CREATE TABLE [Case Metadata]
(

	[Case Code] INT IDENTITY (1, 1) PRIMARY KEY,
	[Case Name] VARCHAR (30) NOT NULL,
	[Application Date] DATE NOT NULL,
	[Decision Date] DATE NOT NULL,
	[Case Reference] VARCHAR (MAX) NOT NULL,
	[Footnote] VARCHAR (MAX) NOT NULL,
	[Link] VARCHAR (MAX) NOT NULL	
);

CREATE TABLE [Political Cases]
(
	[Political Incident] VARCHAR (100) NOT NULL PRIMARY KEY,
	[Case Name] VARCHAR (30) NOT NULL,
    [Negative Denominator] BIT,
    [Positive Denominator] BIT,
	[Political Dummy] AS CASE
		WHEN [Positive Denominator] = 1 THEN 1
		WHEN [Negative Denominator] = 1 THEN -1
		ELSE NULL
	END,
    [Criteria] VARCHAR (MAX) NOT NULL,
    CONSTRAINT [Was it the court or a judge?] CHECK ([Negative Denominator] != [Positive Denominator] AND LEN([Criteria]) > 30),
	CONSTRAINT [Politicization Check] FOREIGN KEY ([Case Name]) REFERENCES [Case Statistics] ([Case Name])
);



--Make this a trigger: 	[Political Incident] VARCHAR (100) FOREIGN KEY REFERENCES [Political Cases of Human Rights Courts]([Political Incident])


In [ ]:
---Autocomplete feature for the "Footnote" column on the "Case Metadata" table---
CREATE FUNCTION ReferenceGenerator (
    @ConventionCall VARCHAR (50),
    @CaseName VARCHAR (30),
    @RespondentCall VARCHAR (30),
    @CaseReference VARCHAR (MAX),
    @DecisionDate DATE,
    @Link VARCHAR (MAX)
)
RETURNS VARCHAR (100)
AS
BEGIN

    DECLARE @AutomatedFootnote VARCHAR (100)
    DECLARE @Convention VARCHAR (50)

    SELECT @Convention = @ConventionCall
    FROM [Human Rights Defenders] AS hr
    INNER JOIN [European Court of Human Rights] AS e ON e.[Respondent] = hr.[Country Name]
    INNER JOIN [Interamerican Court of Human Rights] AS ia ON ia.[Respondent] = hr.[Country Name]
    INNER JOIN [African Court of Human and Peoples' Rights] AS a ON a.[Respondent] = hr.[Country name]
    WHERE @RespondentCall = hr.[Country Name]

    IF @Convention = 'ECHR'
       SET @AutomatedFootnote =  CONCAT (@CaseName, ' v. ', @RespondentCall, ' , ', @CaseReference, ' [', YEAR(@DecisionDate), '] , Eur. Ct. H. R., ', @Link)
    ELSE IF @Convention = 'IACHR'
       SET @AutomatedFootnote =  CONCAT (@CaseName, ' v. ', @RespondentCall, ' , ', @CaseReference, ' [', YEAR(@DecisionDate), '] , Interam. Ct. H. R., ', @Link)
    ELSE IF @Convention = 'ACHPR'
       SET  @AutomatedFootnote = CONCAT (@CaseName, ' v. ', @RespondentCall, ' , ', @CaseReference, ' [', YEAR(@DecisionDate), '] , Afr, Ct. H. P. R., ', @Link)
    ELSE 
        SET @AutomatedFootnote = 'N/A'

    RETURN @AutomatedFootnote
END
;

GO

CREATE FUNCTION RealiseJoins2 (
    @RespondentCall VARCHAR(30)
)
RETURNS VARCHAR(30)
AS
BEGIN
    DECLARE @Result1 VARCHAR(30)
    DECLARE @Result2 VARCHAR(30)
    DECLARE @Result3 VARCHAR(30)

    SELECT @Result1 = @RespondentCall
    FROM [European Court of Human Rights] AS e
    INNER JOIN [Case Metadata] ON [Case Metadata].[Case Name] = e.[Case Name]

    IF @Result1 IS NOT NULL
        RETURN @Result1

    SELECT @Result2 = @RespondentCall
    FROM [Interamerican Court of Human Rights] AS i
    INNER JOIN [Case Metadata] ON [Case Metadata].[Case Name] = i.[Case Name]

    IF @Result2 IS NOT NULL
        RETURN @Result2

    SELECT @Result3 = @RespondentCall
    FROM [African Court of Human and Peoples' Rights] AS a
    INNER JOIN [Case Metadata] ON [Case Metadata].[Case Name] = a.[Case Name]

    IF @Result3 IS NOT NULL
        RETURN @Result3

    RETURN 'N/A'
END;

GO

CREATE TRIGGER [Respondent Autocomplete 1]
ON [European Court of Human Rights]
AFTER INSERT, UPDATE
AS
BEGIN
    IF (
        SELECT Respondent
        FROM [European Court of Human Rights]
    ) IS NOT NULL
        UPDATE [Case Metadata]
        SET [Respondent] = dbo.RealiseJoins2 (
		[Respondent])
END;

GO

CREATE TRIGGER [Footnote Autocomplete 1]
ON [European Court of Human Rights]
AFTER INSERT, UPDATE
AS
BEGIN
   IF (
        SELECT *
        FROM [European Court of Human Rights]
    ) IS NOT NULL
    UPDATE [Case Metaddata]
    SET [@AutomatedFootnote] = ReferenceGenerator ([Human Rights Defenders].[Convention],
    [Case Metadata].[Respondent],
    [Case Metadata].[Case Reference],
    [Case Metadata].[Decision Date],
    [Case Metadata].[Link])
END;

GO

ALTER TABLE [Case Metadata]
ADD [Automated Footnote ECHR] AS dbo.ReferenceGenerator (
    [Human Rights Defenders].[Convention],
    [Case Metadata].[Respondent],
    [Case Metadata].[Case Reference],
    [Case Metadata].[Decision Date],
    [Case Metadata].[Link]);

In [ ]:
---Full case docket junction table---

CREATE TABLE [All - Meta Index] (
[Case Meta Index] INT IDENTITY (1,1) PRIMARY KEY,
[European Case Names] VARCHAR (30),
[African Case Names] VARCHAR (30),
[American Case Names] VARCHAR (30)
);

GO

ALTER TABLE [All - Meta Index]
ADD CONSTRAINT UniqueMeta1 UNIQUE ([European Case Names], [American Case Names], [African Case Names]);

GO


CREATE TRIGGER AutoComplete1
ON [European Court of Human Rights]
AFTER INSERT, UPDATE
AS
BEGIN
    UPDATE [All - Meta Index]
    SET [European Case Names] = (SELECT DISTINCT [Case Name] FROM [European Court of Human Rights])
END;

GO

CREATE TRIGGER AutoComplete2
ON [Interamerican Court of Human Rights]
AFTER INSERT, UPDATE
AS
BEGIN
    UPDATE [All - Meta Index]
    SET [American Case Names] = (SELECT DISTINCT [Case Name] FROM [Interamerican Court of Human Rights])
END;

GO

CREATE TRIGGER AutoComplete3
ON [African Court of Human and Peoples' Rights]
AFTER INSERT, UPDATE
AS
BEGIN
    UPDATE [All - Meta Index]
    SET [African Case Names] = (SELECT DISTINCT [Case Name] FROM [African Court of Human and Peoples' Rights])
END;

In [ ]:
---Connections for the "Case Metadata" component---

CREATE TRIGGER AutoComplete4
ON [All - Meta Index]
AFTER INSERT
AS
BEGIN
    INSERT INTO [Case Metadata] ([Case Name])
    VALUES (
        SELECT DISTINCT [Case Name] 
        FROM [African Court of Human and Peoples' Rights]
        WHERE )
END;